## Decision Tree (classification)

### Gini impurity

 * with $c$ classes:
$$I_G(p)=1 - \sum_{j=1}^c p_j^2$$

In [1]:
import math
from collections import Counter

import numpy as np

In [2]:

def calc_gini(y):
    m = len(y)
    if m == 0:
        return 0
    counts = Counter(y)
    probas = [c/m for c in counts.values()]
    impurity = 1 - sum([p**2 for p in probas])
    return impurity

# testing the function
y = [1, 1, 1, 1, 0, 0, 0]
print(calc_gini(y))

0.48979591836734704


In [3]:
def entropy(y):
    m = len(y)
    counts = Counter(y)
    probas = [c/m for c in counts.values()]
    return -sum([p * math.log(p, 2) for p in probas])

# testing the function
y = [1, 1, 1, 1, 0, 0, 0]
print(entropy(y))

0.9852281360342516


In [4]:
def split_dataset(X, feature_index, threshold):
    left_indices = np.where(X[:, feature_index] < threshold)[0]
    right_indices = np.where(X[:, feature_index] >= threshold)[0]

    return left_indices, right_indices

# testing the function
X = np.array([[-1, 2], [3, 4], [3, 6], [-2, 8]])
feature_index = 0
threshold = 0
left, right = split_dataset(X, feature_index, threshold)
print(left, right)

[0 3] [1 2]


In [5]:
def gini_split(X, y, feature_index, threshold):
    left_indices, right_indices = split_dataset(X, feature_index, threshold)
    left_y, right_y = y[left_indices], y[right_indices]
    m = len(y)
    w_left, w_right = len(left_y) / m, len(right_y) / m
    gini_left = calc_gini(left_y)
    gini_right = calc_gini(right_y)
    gini = (w_left/m) * gini_left + (w_right/m) * gini_right
    return gini

# testing the function
y = np.array([1, 1, 0, 0])
print(gini_split(X, y, feature_index, threshold))
y = np.array([0, 1, 1, 0])
print(gini_split(X, y, feature_index, threshold))

0.125
0.0


In [ ]:
class DecisionTree():
    def __init__(
            self,
            max_depth=None,
            min_samples_split=2,
            criterion='gini'
        ):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape

        if depth >= self.max_depth or n_samples < self.min_samples_split:
            return {
                "leaf": True,
                'value': self._most_common_label(y)
            }
        
        if self.criterion == 'gini':
            impurity_func = calc_gini
        else:
            impurity_func = entropy

        best_feature, best_threshold = self._find_best_split(X, y, n_features)
        if best_feature is None:
            return {
                "leaf": True,
                'value': self._most_common_label(y)
            }
        left, right = split_dataset(X, best_feature, best_threshold)

        return {
            "leaf": False,
            "feature_index": best_feature,
            "threshold": best_threshold,
            "left": self._build_tree(X[left], y[left], depth + 1),
            "right": self._build_tree(X[right], y[right], depth + 1)
        }
        
    def _find_best_split(self, X, y, n_features):
        best_feature = None
        best_threshold = None
        best_impurity = float('inf')

        for feature_index in range(n_features):
            unique_features = np.unique(X[:, feature_index])
            for threshold in unique_features:
                left, right = split_dataset(X, feature_index, threshold)
                if len(left) == 0 or len(right) == 0:
                    continue
                impurity = self._impurity_split(y, left, right)
                if impurity < best_impurity:
                    best_feature = feature_index
                    best_threshold = threshold
                    best_impurity = impurity

        return best_feature, best_threshold
    
    def _impurity_split(self, y, left, right):
        n = len(y)
        n_l, n_r = len(left), len(right)
        p_l, p_r = n_l / n, n_r / n
        if self.criterion == 'gini':
            return p_l * calc_gini(y[left]) + p_r * calc_gini(y[right])
        else:
            return p_l * entropy(y[left]) + p_r * entropy(y[right])
    
    def _most_common_label(self, y):
        return Counter(y).most_common(1)[0][0]
    
    def predict(self, X):
        predictions = [
            self._predict_sample(inputs, self.tree) for inputs in X
        ]
        return np.array(predictions)
    
    def _predict_sample(self, x, tree):
        if tree['leaf']:
            return tree['value']
        feature_value = x[tree['feature_index']]
        if feature_value < tree['threshold']:
            return self._predict_sample(x, tree['left'])
        else:
            return self._predict_sample(x, tree['right'])


In [7]:
# testing the DecisionTree class using the iris dataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# set random seed for reproducibility
np.random.seed(100)
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tree = DecisionTree(max_depth=3)
tree.fit(X_train, y_train)
predictions = tree.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

Accuracy: 0.9666666666666667
